In [ ]:
# mount drive 
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import 
import numpy as np
from scipy.stats import norm
import pandas as pd

from plotly import graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

pio.templates.default = "plotly_white"


In [ ]:
# prediction
import statsmodels.api as sm 

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV

from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
# loading data
data = pd.read_table("/content/drive/My Drive/Colab Notebooks/project stat/very_final_table_paper.csv", sep=";", header=0)

In [ ]:
data

,Unnamed: 0,level_0,index,consultation_id,weight,height,total_fall_count,drugs,age,gender,diagnostic,patient_id,FEATURE_zero_crossing_SPD_ML_opened_eyes,FEATURE_zero_crossing_SPD_AP_opened_eyes,FEATURE_frequency_mode_Power_Spectrum_Density_ML_opened_eyes,FEATURE_frequency_mode_Power_Spectrum_Density_AP_opened_eyes,FEATURE_total_power_Power_Spectrum_Density_ML_opened_eyes,FEATURE_total_power_Power_Spectrum_Density_AP_opened_eyes,FEATURE_power_frequency_50_Power_Spectrum_Density_ML_opened_eyes,FEATURE_power_frequency_50_Power_Spectrum_Density_AP_opened_eyes,FEATURE_power_frequency_95_Power_Spectrum_Density_ML_opened_eyes,FEATURE_power_frequency_95_Power_Spectrum_Density_AP_opened_eyes,FEATURE_centroid_frequency_Power_Spectrum_Density_ML_opened_eyes,FEATURE_centroid_frequency_Power_Spectrum_Density_AP_opened_eyes,FEATURE_frequency_dispersion_Power_Spectrum_Density_ML_opened_eyes,FEATURE_frequency_dispersion_Power_Spectrum_Density_AP_opened_eyes,FEATURE_energy_content_0_05_Power_Spectrum_Density_ML_opened_eyes,FEATURE_energy_content_0_05_Power_Spectrum_Density_AP_opened_eyes,FEATURE_energy_content_05_2_Power_Spectrum_Density_ML_opened_eyes,FEATURE_energy_content_05_2_Power_Spectrum_Density_AP_opened_eyes,FEATURE_energy_content_2_inf_Power_Spectrum_Density_ML_opened_eyes,FEATURE_energy_content_2_inf_Power_Spectrum_Density_AP_opened_eyes,FEATURE_frequency_quotient_Power_Spectrum_Density_ML_opened_eyes,FEATURE_frequency_quotient_Power_Spectrum_Density_AP_opened_eyes,FEATURE_principal_sway_direction_ML_AND_AP_opened_eyes,FEATURE_confidence_ellipse_area_ML_AND_AP_opened_eyes,FEATURE_maximum_value_ML_opened_eyes,FEATURE_maximum_value_AP_opened_eyes,FEATURE_maximum_value_Radius_opened_eyes,FEATURE_mean_velocity_ML_opened_eyes,...,FEATURE_RMS_ML_closed_eyes,FEATURE_RMS_AP_closed_eyes,FEATURE_RMS_Radius_closed_eyes,FEATURE_amplitude_ML_closed_eyes,FEATURE_amplitude_AP_closed_eyes,FEATURE_amplitude_ML_AND_AP_closed_eyes,FEATURE_sway_length_ML_closed_eyes,FEATURE_sway_length_AP_closed_eyes,FEATURE_sway_length_ML_AND_AP_closed_eyes,FEATURE_Coefficient_sway_direction_ML_AND_AP_closed_eyes,FEATURE_Quotient_both_direction_ML_AND_AP_closed_eyes,FEATURE_planar_deviation_ML_AND_AP_closed_eyes,FEATURE_peak_velocity_all_SPD_ML_closed_eyes,FEATURE_peak_velocity_all_SPD_AP_closed_eyes,FEATURE_peak_velocity_pos_SPD_ML_closed_eyes,FEATURE_peak_velocity_pos_SPD_AP_closed_eyes,FEATURE_peak_velocity_neg_SPD_ML_closed_eyes,FEATURE_peak_velocity_neg_SPD_AP_closed_eyes,FEATURE_mean_peak_Sway_Density_closed_eyes,FEATURE_mean_distance_peak_Sway_Density_closed_eyes,FEATURE_sway_area_per_second_ML_AND_AP_closed_eyes,FEATURE_length_over_area_ML_AND_AP_closed_eyes,FEATURE_phase_plane_parameters_ML_closed_eyes,FEATURE_phase_plane_parameters_AP_closed_eyes,FEATURE_mean_frequency_ML_closed_eyes,FEATURE_mean_frequency_AP_closed_eyes,FEATURE_mean_frequency_ML_AND_AP_closed_eyes,FEATURE_fractal_dimension_pd_ML_AND_AP_closed_eyes,FEATURE_fractal_dimension_cc_ML_AND_AP_closed_eyes,FEATURE_fractal_dimension_ce_ML_AND_AP_closed_eyes,FEATURE_short_time_diffusion_Diffusion_ML_closed_eyes,FEATURE_long_time_diffusion_Diffusion_ML_closed_eyes,FEATURE_critical_time_Diffusion_ML_closed_eyes,FEATURE_critical_displacement_Diffusion_ML_closed_eyes,FEATURE_long_time_scaling_Diffusion_ML_closed_eyes,FEATURE_short_time_diffusion_Diffusion_AP_closed_eyes,FEATURE_long_time_diffusion_Diffusion_AP_closed_eyes,FEATURE_critical_time_Diffusion_AP_closed_eyes,FEATURE_critical_displacement_Diffusion_AP_closed_eyes,FEATURE_long_time_scaling_Diffusion_AP_closed_eyes
0,0,233,384,2498,59.82,155,0.0,['-1'],76,female,['Proprioceptif'],-6756443073212183619,151,80,0.097656,0.097656,0.156399,1.514931,0.097656,0.195312,0.976562,0.585938,0.711968,0.335636,0.880688,0.737552,0.128365,1.424691,0.020332,0.072725,0.003427,0.003357,0.020154,0.001981,3.086282,1.251098,0.470574,0.893351,0.900527,0.439079,...,0.169118,0.381648,0.417440,0.995326,2.165980,2.258467,8.777444,14.048024,17.931296,0.024112,0.459527,0.417440

In [ ]:
# transform drugs, diagnostic, gender to a binary features

# 1 : if drugs 1 otherwise 0
def create_drugs_col(entry):
  if entry == "['-1']":
    return 0

  return 1


# 1 : if diagnostic 1 otherwise 0
def create_diagnostic_col(entry):
  if entry == "['Aucun']":
    return 0

  return 1

data["take_drugs"] = data["drugs"].apply(create_drugs_col)
data["has_symptom"] = data["diagnostic"].apply(create_diagnostic_col)
data["sex"] = data["gender"].map({"female": 0, "male": 1})

In [ ]:
# creating new column faller / none faller
data["faller"] = data["total_fall_count"].apply(lambda entry: 1 if entry > 1 else 0)

Y_train = data["faller"]

In [ ]:
# scoring the model
def computeScore(model, Y, X):
  treshold = 0.3
  prediction = [1 if prob > treshold else 0 for prob in  model.predict(X)]

  
  return accuracy_score(Y, prediction)

In [ ]:
# features by cluster
patient = [
           "height",
           "weight",
           "sex",
           "age",
           "take_drugs",
           "has_symptom"
]


position = [
            "FEATURE_confidence_ellipse_area_ML_AND_AP_opened_eyes",
            "FEATURE_maximum_value_ML_opened_eyes","FEATURE_maximum_value_AP_opened_eyes",
            "FEATURE_maximum_value_Radius_opened_eyes","FEATURE_mean_distance_ML_opened_eyes",
            "FEATURE_mean_distance_AP_opened_eyes",
            "FEATURE_mean_distance_Radius_opened_eyes",
            "FEATURE_RMS_ML_opened_eyes",
            "FEATURE_RMS_AP_opened_eyes",
            "FEATURE_RMS_Radius_opened_eyes",
            "FEATURE_amplitude_ML_opened_eyes",
            "FEATURE_amplitude_AP_opened_eyes",
            "FEATURE_amplitude_ML_AND_AP_opened_eyes",
            "FEATURE_sway_length_ML_opened_eyes",
            "FEATURE_sway_length_AP_opened_eyes",
            "FEATURE_sway_length_ML_AND_AP_opened_eyes",
            "FEATURE_Coefficient_sway_direction_ML_AND_AP_opened_eyes",
                      
            'FEATURE_maximum_value_AP_closed_eyes',
            'FEATURE_maximum_value_Radius_closed_eyes', 
            'FEATURE_mean_distance_ML_closed_eyes', 
            'FEATURE_mean_distance_AP_closed_eyes', 
            'FEATURE_mean_distance_Radius_closed_eyes', 
            'FEATURE_RMS_ML_closed_eyes', 
            'FEATURE_RMS_AP_closed_eyes', 
            'FEATURE_RMS_Radius_closed_eyes',
            'FEATURE_amplitude_ML_closed_eyes', 
            'FEATURE_amplitude_AP_closed_eyes', 
            'FEATURE_amplitude_ML_AND_AP_closed_eyes', 
            'FEATURE_sway_length_ML_closed_eyes', 
            'FEATURE_sway_length_AP_closed_eyes', 
            'FEATURE_sway_length_ML_AND_AP_closed_eyes', 
            'FEATURE_length_over_area_ML_AND_AP_closed_eyes', 
            'FEATURE_fractal_dimension_pd_ML_AND_AP_closed_eyes',
          
            'FEATURE_fractal_dimension_pd_ML_AND_AP_opened_eyes',
            'FEATURE_length_over_area_ML_AND_AP_opened_eyes',
            'FEATURE_maximum_value_ML_closed_eyes',
            'FEATURE_confidence_ellipse_area_ML_AND_AP_closed_eyes',
            'FEATURE_length_over_area_ML_AND_AP_opened_eyes'
            ]

dynamic = [
        
           "FEATURE_zero_crossing_SPD_ML_opened_eyes",
           "FEATURE_zero_crossing_SPD_AP_opened_eyes",
           "FEATURE_principal_sway_direction_ML_AND_AP_opened_eyes",
           "FEATURE_mean_velocity_ML_opened_eyes",
           "FEATURE_mean_velocity_AP_opened_eyes",
           "FEATURE_mean_velocity_ML_AND_AP_opened_eyes",
           "FEATURE_Coefficient_sway_direction_ML_AND_AP_opened_eyes",
           "FEATURE_planar_deviation_ML_AND_AP_opened_eyes",
           "FEATURE_peak_velocity_all_SPD_ML_opened_eyes",
           "FEATURE_peak_velocity_all_SPD_AP_opened_eyes",
           "FEATURE_peak_velocity_pos_SPD_ML_opened_eyes",
           "FEATURE_peak_velocity_pos_SPD_AP_opened_eyes",
 
           'FEATURE_mean_velocity_ML_closed_eyes', 
           'FEATURE_mean_velocity_AP_closed_eyes', 
           'FEATURE_mean_velocity_ML_AND_AP_closed_eyes', 
           'FEATURE_Coefficient_sway_direction_ML_AND_AP_closed_eyes', 
           'FEATURE_Quotient_both_direction_ML_AND_AP_closed_eyes', 
           'FEATURE_planar_deviation_ML_AND_AP_closed_eyes', 
           'FEATURE_peak_velocity_all_SPD_ML_closed_eyes', 
           'FEATURE_peak_velocity_all_SPD_AP_closed_eyes', 
           'FEATURE_peak_velocity_pos_SPD_ML_closed_eyes', 
           'FEATURE_peak_velocity_pos_SPD_AP_closed_eyes', 
           'FEATURE_peak_velocity_neg_SPD_ML_closed_eyes', 
           'FEATURE_peak_velocity_neg_SPD_AP_closed_eyes', 
           'FEATURE_mean_peak_Sway_Density_closed_eyes', 
           'FEATURE_mean_distance_peak_Sway_Density_closed_eyes', 
           'FEATURE_sway_area_per_second_ML_AND_AP_closed_eyes', 
           'FEATURE_phase_plane_parameters_ML_closed_eyes', 
           'FEATURE_phase_plane_parameters_AP_closed_eyes', 
           'FEATURE_fractal_dimension_cc_ML_AND_AP_closed_eyes', 
           'FEATURE_fractal_dimension_ce_ML_AND_AP_closed_eyes',
      
           'FEATURE_mean_frequency_ML_closed_eyes', 
           'FEATURE_mean_frequency_AP_closed_eyes', 
           'FEATURE_mean_frequency_ML_AND_AP_closed_eyes',
        
           'FEATURE_mean_frequency_ML_opened_eyes',
           'FEATURE_mean_frequency_AP_opened_eyes',
           'FEATURE_mean_frequency_ML_AND_AP_opened_eyes',
           'FEATURE_peak_velocity_neg_SPD_ML_opened_eyes',
           'FEATURE_peak_velocity_neg_SPD_AP_opened_eyes',
           'FEATURE_fractal_dimension_cc_ML_AND_AP_opened_eyes',
           'FEATURE_fractal_dimension_ce_ML_AND_AP_opened_eyes',
           'FEATURE_mean_peak_Sway_Density_opened_eyes',
           'FEATURE_phase_plane_parameters_AP_opened_eyes',
           'FEATURE_phase_plane_parameters_ML_opened_eyes',
           'FEATURE_sway_area_per_second_ML_AND_AP_opened_eyes',
           'FEATURE_zero_crossing_SPD_AP_closed_eyes',
           'FEATURE_zero_crossing_SPD_ML_closed_eyes',
           'FEATURE_principal_sway_direction_ML_AND_AP_closed_eyes'
           ]

frequency = [
             "FEATURE_frequency_mode_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_frequency_mode_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_total_power_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_total_power_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_power_frequency_50_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_power_frequency_50_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_power_frequency_95_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_power_frequency_95_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_centroid_frequency_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_centroid_frequency_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_frequency_dispersion_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_frequency_dispersion_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_energy_content_0_05_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_energy_content_0_05_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_energy_content_05_2_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_energy_content_05_2_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_energy_content_2_inf_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_energy_content_2_inf_Power_Spectrum_Density_AP_opened_eyes",
             "FEATURE_frequency_quotient_Power_Spectrum_Density_ML_opened_eyes",
             "FEATURE_frequency_quotient_Power_Spectrum_Density_AP_opened_eyes",
        
             'FEATURE_frequency_mode_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_frequency_mode_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_power_frequency_50_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_power_frequency_50_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_power_frequency_95_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_power_frequency_95_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_centroid_frequency_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_centroid_frequency_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_frequency_dispersion_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_frequency_dispersion_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_energy_content_0_05_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_energy_content_0_05_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_energy_content_05_2_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_energy_content_05_2_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_energy_content_2_inf_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_energy_content_2_inf_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_frequency_quotient_Power_Spectrum_Density_ML_closed_eyes',
             'FEATURE_frequency_quotient_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_total_power_Power_Spectrum_Density_AP_closed_eyes',
             'FEATURE_total_power_Power_Spectrum_Density_ML_closed_eyes'
             ]

stochastic = [
     
              'FEATURE_short_time_diffusion_Diffusion_ML_closed_eyes', 
              'FEATURE_long_time_diffusion_Diffusion_ML_closed_eyes', 
              'FEATURE_critical_time_Diffusion_ML_closed_eyes', 
              'FEATURE_long_time_scaling_Diffusion_ML_closed_eyes', 
              'FEATURE_short_time_diffusion_Diffusion_AP_closed_eyes', 
              'FEATURE_long_time_diffusion_Diffusion_AP_closed_eyes', 
              'FEATURE_critical_time_Diffusion_AP_closed_eyes', 
              'FEATURE_long_time_scaling_Diffusion_AP_closed_eyes', 
              'FEATURE_critical_displacement_Diffusion_ML_closed_eyes', 
              'FEATURE_critical_displacement_Diffusion_AP_closed_eyes',
  
              'FEATURE_short_time_diffusion_Diffusion_ML_opened_eyes',
              'FEATURE_long_time_diffusion_Diffusion_ML_opened_eyes',
              'FEATURE_critical_time_Diffusion_ML_opened_eyes',
              'FEATURE_critical_displacement_Diffusion_ML_opened_eyes',
              'FEATURE_long_time_scaling_Diffusion_ML_opened_eyes',
              'FEATURE_short_time_diffusion_Diffusion_AP_opened_eyes',
              'FEATURE_long_time_diffusion_Diffusion_AP_opened_eyes',
              'FEATURE_critical_time_Diffusion_AP_opened_eyes',
              'FEATURE_critical_displacement_Diffusion_AP_opened_eyes',
              'FEATURE_long_time_scaling_Diffusion_AP_opened_eyes'
              ]

patient = pd.Index(patient).unique()
position = pd.Index(position).unique()
dynamic = pd.Index(dynamic).unique()
frequency = pd.Index(frequency).unique()
stochastic = pd.Index(stochastic).unique()

clusters = dict(
    patient=patient,
    position= position,
    dynamic=dynamic,
    frequency=frequency,
    stochastic=stochastic
)

In [ ]:
features_scores = pd.DataFrame(columns=["feature", "cluster", "logistic_regression_score"])

# consider only variables related to stabilogram
for cluster_name in clusters.keys():
  for feat in clusters[cluster_name]:

    # select the vars
    Y_train = data["faller"]
    X_train = data[feat]

    # build model
    log_reg = sm.Logit(Y_train, X_train).fit(disp=False)
    score = computeScore(log_reg, Y_train, X_train)

    features_scores = features_scores.append({"feature": feat, "cluster": cluster_name, "logistic_regression_score": score}, ignore_index=True)

In [ ]:
fig = px.box(features_scores, y="logistic_regression_score")

fig.show()

In [ ]:
# sort features by their score
features_scores.sort_values(by="logistic_regression_score", ascending=False, inplace=True)

In [ ]:
features_scores

,feature,cluster,logistic_regression_score
37,FEATURE_length_over_area_ML_AND_AP_closed_eyes,position,0.65
73,FEATURE_fractal_dimension_ce_ML_AND_AP_closed_...,dynamic,0.61
5,has_symptom,patient,0.61
38,FEATURE_fractal_dimension_pd_ML_AND_AP_closed_...,position,0.60
88,FEATURE_zero_crossing_SPD_AP_closed_eyes,dynamic,0.60
...,...,...,...
53,FEATURE_peak_velocity_pos_SPD_ML_opened_eyes,dynamic,0.29
63,FEATURE_peak_velocity_pos_SPD_ML_closed_eyes,dynamic,0.29
69,FEATURE_sway_area_per_second_ML_AND_AP_closed_...,dynamic,0.29
51,FEATURE_peak_velocity_all_SPD_ML_opened_eyes,dynamic,0.28


In [ ]:
# features_scores["feature"] = features_scores["feature"].apply(lambda entry: entry.replace("FEATURE_", ""))

# features_scores.to_latex(index=False)

In [ ]:
# select the 3rd quartile
quartile3 = 0.47

features_scores = features_scores[features_scores["logistic_regression_score"] >= quartile3]
features_scores.sort_values(by="logistic_regression_score", ascending=False, inplace=True)

len(features_scores)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



40

In [ ]:
colors = features_scores["cluster"].map({
    "position": "#636EF1", 
    "dynamic": "#EF553B", 
    "frequency": "#00CC96", 
    "stochastic": "#AB63FA", 
    "patient": "#FFA15A"
    })[:20]

size = [15 for i in range(len(features_scores))][:20]

fig = go.Figure(data=[
                      go.Scatter(
                          x=features_scores["feature"][:20], 
                          y=features_scores["logistic_regression_score"][:20], 
                          mode="markers+lines",
                          marker=dict(
                              color=colors,
                              size=size,
                              ),
                          line=dict(
                            shape="spline",
                            width=1  
                          ),
                          text=features_scores["cluster"][:20]
                      )
])



fig.update_layout(dict(
    height=800,
    title="$$Most \ importante \ features \ (3^{rd} quartile)$$",
    yaxis_title="score"
))

fig.update_xaxes(tickfont=dict(size=10), tickangle=45)

fig.show()

In [ ]:
features_scores.to_latex(index=False)

'\\begin{tabular}{llr}\n\\toprule\n                                           feature &    cluster &  logistic\\_regression\\_score \\\\\n\\midrule\n    FEATURE\\_length\\_over\\_area\\_ML\\_AND\\_AP\\_closed\\_eyes &   position &                       0.65 \\\\\n                                       has\\_symptom &    patient &                       0.61 \\\\\n FEATURE\\_fractal\\_dimension\\_ce\\_ML\\_AND\\_AP\\_closed\\_... &    dynamic &                       0.61 \\\\\n FEATURE\\_fractal\\_dimension\\_pd\\_ML\\_AND\\_AP\\_closed\\_... &   position &                       0.60 \\\\\n          FEATURE\\_zero\\_crossing\\_SPD\\_AP\\_closed\\_eyes &    dynamic &                       0.60 \\\\\n FEATURE\\_fractal\\_dimension\\_cc\\_ML\\_AND\\_AP\\_closed\\_... &    dynamic &                       0.60 \\\\\n      FEATURE\\_mean\\_frequency\\_ML\\_AND\\_AP\\_closed\\_eyes &    dynamic &                       0.59 \\\\\n             FEATURE\\_mean\\_frequency\\_AP\\_closed\\_eyes &    

In [ ]:
features_scores["feature"] = features_scores["feature"].apply(lambda entry: entry.replace("FEATURE_", ""))
features_scores.to_latex(index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



'\\begin{tabular}{llr}\n\\toprule\n                                           feature &    cluster &  logistic\\_regression\\_score \\\\\n\\midrule\n            length\\_over\\_area\\_ML\\_AND\\_AP\\_closed\\_eyes &   position &                       0.65 \\\\\n                                       has\\_symptom &    patient &                       0.61 \\\\\n        fractal\\_dimension\\_ce\\_ML\\_AND\\_AP\\_closed\\_eyes &    dynamic &                       0.61 \\\\\n        fractal\\_dimension\\_pd\\_ML\\_AND\\_AP\\_closed\\_eyes &   position &                       0.60 \\\\\n                  zero\\_crossing\\_SPD\\_AP\\_closed\\_eyes &    dynamic &                       0.60 \\\\\n        fractal\\_dimension\\_cc\\_ML\\_AND\\_AP\\_closed\\_eyes &    dynamic &                       0.60 \\\\\n              mean\\_frequency\\_ML\\_AND\\_AP\\_closed\\_eyes &    dynamic &                       0.59 \\\\\n                     mean\\_frequency\\_AP\\_closed\\_eyes &    dynamic &     